In [5]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import PIL
import os

# Loading and Cleaning the data

In [6]:
files = "/Users/Tim/Downloads/hotdog-nothotdog/"

In [7]:
train_dir = os.path.join(files, 'train')
validation_dir = os.path.join(files, 'test')

In [14]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    image_size=(224, 224),
    batch_size=32,
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset="training"
)

Found 3000 files belonging to 2 classes.
Using 2400 files for training.


In [15]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    image_size=(224, 224),
    batch_size=32,
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset="validation"
)

Found 3000 files belonging to 2 classes.
Using 600 files for validation.


In [16]:
class_names = train_ds.class_names

In [17]:
class_names

['hotdog', 'nothotdog']

In [35]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [36]:
nomralization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)

In [37]:
normalized_ds = train_ds.map(lambda x, y: (nomralization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))

2023-03-31 15:45:20.436950: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [20]:
num_classes = 2

# Building the model

In [43]:
model = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape=(224, 224, 3)),
    tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

In [44]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [45]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_6 (Rescaling)     (None, 224, 224, 3)       0         
                                                                 
 conv2d_15 (Conv2D)          (None, 224, 224, 16)      448       
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 112, 112, 16)     0         
 g2D)                                                            
                                                                 
 conv2d_16 (Conv2D)          (None, 112, 112, 32)      4640      
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 56, 56, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_17 (Conv2D)          (None, 56, 56, 64)       

In [ ]:
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=100
)